In [53]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from datetime import datetime, timedelta
import time


import warnings
warnings.filterwarnings('ignore')

In [54]:
def single_search(url, driver, Snapshot_date, checking_date_type, checkout_date_type):
    
    errors = 0
    hotels = []
    index = 1
    
    driver.get(url)
    text_box = driver.find_elements(by= By.CLASS_NAME , value='uitk-card-content-section')
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
        
    for t_box in text_box:
        try:
            hotel = {'Snapshot' : None ,
                     'Index':None,
                     'Hotel Name':None,
                     'TTT':None,
                     'LOS':None, 
                     'Grade': None, 
                     'Curr Price per Night': None, 
                     'Former Price per Night': None}
            
            name = t_box.find_element(by= By.CLASS_NAME , value="uitk-heading").text
            score = t_box.find_element(By.CLASS_NAME, 'uitk-spacing-padding-inlineend-half').text
            price = t_box.find_element(By.CSS_SELECTOR, "div[data-test-id='price-summary-message-line']").text.split('$')
          
            hotel['Snapshot'] = Snapshot_date
            hotel['Index'] = index
            hotel['Hotel Name'] = name
            hotel['TTT'] = (checking_date_type - Snapshot_date).days
            hotel['LOS'] = (checkout_date_type - checking_date_type).days
            hotel['Grade'] = score 
            hotel['Curr Price per Night'] = price[-1]
            if len(price) > 3:
                hotel['Former Price per Night'] = price[1].split('\n')[0]
            hotels.append(hotel)
            
            index = index + 1
        except:
            errors = errors + 1
    return pd.DataFrame.from_dict(hotels), errors

In [55]:
#Allows us to run on different computers without a problem

path ={'roy' :'C:\Windows\chromdriver.exe' , 'yarin': 0 , 'yair':'C:\Windows\chromdriver.exe' }


In [56]:
# Get today's date
now = datetime.now()
Snapshot_date =now
# Get end date 35 days from now
end_date = now + timedelta(days=35)

# Create a list of all dates between today and end date
date_list = []
while now <= end_date:
    date_list.append(now.strftime('%Y-%m-%d'))
    now += timedelta(days=1)


#Creates a list whose every member is a list of checking and checkout time

combinations_list =[]
for i in range(1,len(date_list)-5):
    for j in range(1,6):
        combinations = [date_list[i],date_list[i+j]]
        combinations_list.append(combinations)

print(combinations_list , len(combinations_list)) 

[['2023-02-08', '2023-02-09'], ['2023-02-08', '2023-02-10'], ['2023-02-08', '2023-02-11'], ['2023-02-08', '2023-02-12'], ['2023-02-08', '2023-02-13'], ['2023-02-09', '2023-02-10'], ['2023-02-09', '2023-02-11'], ['2023-02-09', '2023-02-12'], ['2023-02-09', '2023-02-13'], ['2023-02-09', '2023-02-14'], ['2023-02-10', '2023-02-11'], ['2023-02-10', '2023-02-12'], ['2023-02-10', '2023-02-13'], ['2023-02-10', '2023-02-14'], ['2023-02-10', '2023-02-15'], ['2023-02-11', '2023-02-12'], ['2023-02-11', '2023-02-13'], ['2023-02-11', '2023-02-14'], ['2023-02-11', '2023-02-15'], ['2023-02-11', '2023-02-16'], ['2023-02-12', '2023-02-13'], ['2023-02-12', '2023-02-14'], ['2023-02-12', '2023-02-15'], ['2023-02-12', '2023-02-16'], ['2023-02-12', '2023-02-17'], ['2023-02-13', '2023-02-14'], ['2023-02-13', '2023-02-15'], ['2023-02-13', '2023-02-16'], ['2023-02-13', '2023-02-17'], ['2023-02-13', '2023-02-18'], ['2023-02-14', '2023-02-15'], ['2023-02-14', '2023-02-16'], ['2023-02-14', '2023-02-17'], ['2023-02

In [57]:
# for dev
combinations_list = [combinations_list[0]]
# END - for dev

driver = webdriver.Chrome(path['yair'])

city = "New%20York%20%28and%20vicinity%29%2C%20New%20York%2C%20United%20States%20of%20America"
city_latlong = 'latLong=40.75668%2C-73.98647'
city_region_id = '178293'
numOfAdults=2
numOfChild=0
numOfRooms=1
expedia_df = pd.DataFrame(columns=['Snapshot', 'Index', 'Hotel Name', 'TTT', 'LOS', 'Grade', 'Curr Price per Night', 'Former Price per Night'])
for comb in combinations_list:
    checking_date_type = datetime.strptime(comb[0], '%Y-%m-%d')
    checkout_date_type = datetime.strptime(comb[1], '%Y-%m-%d')
    checking_date = comb[0].split('-')
    checkout_date = comb[1].split('-')
    checking_year = checking_date[0]
    checking_month = checking_date[1]
    checking_monthday =checking_date[2]
    checkout_year= checkout_date[0]
    checkout_month= checkout_date[1]
    checkout_monthday= checkout_date[2]
    url = f"https://www.expedia.com/Hotel-Search?adults={numOfAdults}&d1={comb[0]}&d2={comb[1]}&destination={city}&endDate={comb[1]}&latLong={city_latlong}&regionId={city_region_id}&rooms={numOfRooms}&semdtl=&sort=RECOMMENDED&startDate={comb[0]}&theme=&useRewards=false&userIntent="
    single_search_df, err = single_search(url, driver , Snapshot_date, checking_date_type, checkout_date_type)
    expedia_df = pd.concat([expedia_df, single_search_df])
    print(single_search_df)
    print(err)
#driver.quit()

                     Snapshot  Index  \
0  2023-02-07 02:02:07.614005      1   
1  2023-02-07 02:02:07.614005      2   
2  2023-02-07 02:02:07.614005      3   
3  2023-02-07 02:02:07.614005      4   
4  2023-02-07 02:02:07.614005      5   
5  2023-02-07 02:02:07.614005      6   
6  2023-02-07 02:02:07.614005      7   
7  2023-02-07 02:02:07.614005      8   
8  2023-02-07 02:02:07.614005      9   
9  2023-02-07 02:02:07.614005     10   
10 2023-02-07 02:02:07.614005     11   
11 2023-02-07 02:02:07.614005     12   
12 2023-02-07 02:02:07.614005     13   
13 2023-02-07 02:02:07.614005     14   
14 2023-02-07 02:02:07.614005     15   
15 2023-02-07 02:02:07.614005     16   
16 2023-02-07 02:02:07.614005     17   
17 2023-02-07 02:02:07.614005     18   

                                           Hotel Name  TTT  LOS Grade  \
0     TownePlace Suites by Marriott New York Brooklyn    0    1   8.6   
1   Element New York Times Square West, a Marriott...    0    1   7.8   
2                   

In [58]:
expedia_df

,Snapshot,Index,Hotel Name,TTT,LOS,Grade,Curr Price per Night,Former Price per Night
0,2023-02-07 02:02:07.614005,1,TownePlace Suites by Marriott New York Brooklyn,0,1,8.6,189,None
1,2023-02-07 02:02:07.614005,2,"Element New York Times Square West, a Marriott...",0,1,7.8,94,105
2,2023-02-07 02:02:07.614005,3,Truss Hotel Times Square,0,1,7.8,116,129
3,2023-02-07 02:02:07.614005,4,Park Central Hotel New York,0,1,8.0,98,109
4,2023-02-07 02:02:07.614005,5,Pod Times Square,0,1,8.6,89,None
5,2023-02-07 02:02:07.614005,6,The Belvedere Hotel,0,1,8.6,109,None
6,2023-02-07 02:02:07.614005,7,The MUSE New York,0,1,8.8,342,None
7,2023-02-07 02:02:07.614005,8,Hyatt Place New York City/Times Square,0,1,7.8,119,None
8,2023-02-07 02:02:07.614005,9,Four Points By Sheraton New York Downtown,0,1,7.4,118,139
9,2023-02-07 02:02:07.614005,10,"Holiday Inn New York City - Times Square, an I...",0,1,8.0,89,None


In [59]:
len(expedia_df)

18